# Projet maison n° 3

In [1]:
# imports
import pandas as pd
from os import walk
import re
import numpy as np

## 1. US baby names

On va s'intéresser au dataset **National data** de la SSA : https://www.ssa.gov/oact/babynames/limits.html

1. Télécharger le dataset des prénoms US : https://www.ssa.gov/oact/babynames/names.zip

Lire la documentation associée.

2. Implémenter une fonction Python qui produit un unique DataFrame avec tous les fichiers en utilisant pandas (par ex. fonction "concat" ou méthode "append"), pas de bash :)

Ordre et noms des colonnes : 'year', 'name', 'gender', 'births'

Le DataFrame doit être trié selon l'année croissante puis selon l'ordre défini par les différents fichiers (voir la documentation du dataset).

In [2]:
# names us
def df_names_us():
    df_path = "./names/"
    df_list = []
    (_, _, filenames) = next(walk(df_path))
    m = re.compile(r"[0-9]+")
    for file in filenames:
        if file.endswith(".txt"):
            tmp_df = pd.read_csv(df_path + file, header=None, names=['name','gender', 'births'])
            y = m.search(file)
            if y:
                tmp_df["year"] = int(y.group(0))
            else:
                tmp_df["year"] = 0
            tmp_df = tmp_df.reindex(columns=['year','name', 'gender', 'births'])
            df_list.append(tmp_df)
    df = pd.concat(df_list, axis=0)
    # Each file is sorted first on sex and then on number of occurrences in descending order.
    # When there is a tie on the number of occurrences, names are listed in alphabetical order.
    df = df.sort_values(by=["year", "gender", "births", "name"], ascending=[True, True, False, True])
    df = df.reset_index(drop=True)
    return df


## 2. Prénoms français

On va s'intéresser au dataset **Fichiers France hors Mayotte** de l'INSEE :  https://www.insee.fr/fr/statistiques/2540004/

L'idée est de charger les données et ensuite de les conformer au DataFrame des prénoms US. Ainsi, toute manipulation sur le DataFrame des prénoms US pourra être directement réutilisée avec le DataFrame des prénoms français.
 
1. Télécharger le dataset des prénoms français : https://www.insee.fr/fr/statistiques/fichier/2540004/nat2020_csv.zip


Lire la documentation, ça peut être utile...
 
2. Implémenter une fonction Python qui produit un DataFrame avec les prénoms français en prenant le DataFrame des prénoms US comme modèle :
 
 - Même ordre et mêmes noms des colonnes : year, name, gender, births
 - Mêmes dtypes pour les colonnes
 - Mêmes valeurs pour la colonne 'gender'
 - Seuls les prénoms de 2 caractères et plus sont conservés
 - La casse des prénoms doit être identique : initiales en majuscule, autres lettres en minuscule
 - Les lignes avec des données inutilisables doivent être supprimées
 - Les données sont triées à l'identique : années (↑), puis gender (↑), puis births (↓) et enfin name (↑)
 - L'index du DataFrame doit aller de 0 à N-1

In [3]:
# names fr
def df_names_fr():
    df = pd.read_csv("nat2020.csv", header=0, sep=";", names=['gender', 'name', 'year', 'births'], keep_default_na=False,
                     dtype={"gender":'int', "name":'string', "year":"string", "births":"int"})    
    # Modalité : 1 pour masculin, 2 pour féminin 
    df["gender"] = df["gender"].map({1: "M", 2: "F"})
    # même ordre pour les colonnes
    df = df.reindex(columns=['year', 'name', 'gender', 'births'])
    # prénoms 2 chars ou +
    df = df.loc[df["name"].str.len() >= 2]
    # Première lettre en majuscule et le reste en minuscule
    df["name"] = df["name"].str.capitalize()
    # on drop les lignes avec valeur "_prenoms_rares" à la place du nom
    df = df.loc[df["name"] != "_prenoms_rares"]
    # on drop les années non définies
    df = df.loc[df["year"] != "XXXX"]
    # on convertit les années en nombre entier
    df["year"] = df["year"].astype(int)
    # triage identique aux prénoms US
    df = df.sort_values(by=["year", "gender", "births", "name"], ascending=[True, True, False, True])
    # reset index
    df = df.reset_index(drop=True)
    return df

## 3. Taux de change

On va s'intéresser au dataset des cours des devises de la Banque de France :  http://webstat.banque-france.fr/fr/#/downloading

L'idée est de charger les données, de les nettoyer et de pouvoir accéder aux cours de certaines devises à partir de leur code ISO3.
 
1. Utiliser le dataset des taux de change fourni.


2. Implémenter une fonction qui produit un DataFrame avec les taux de change par date pour une liste de codes ISO3 de devises passée en argument. L'index du DataFrame doit correspondre aux dates (voir la fonction pd.to_datetime() avec le format '%d/%m/%Y'). Les colonnes du DataFrame doivent correspondre aux devises.

In [4]:
# taux de change
def df_taux_change(devises):
    filename = "Webstat_Export_20210929.csv"
    # load le dataset
    df = pd.read_csv(filename, sep=";", header=None)
    # transpose pour avoir le Titre en colonne
    df_t = df.T
    # on extrait le code iso3 de la colonne Titre
    df_t[["iso3"]] = df_t[0].str.extract("\((\w+)\)").astype(str)
    df_t["iso3"][0] = "date"

    # on remet le dataset dans le sens initial
    df = df_t.T.reset_index(drop=True)
    # les codes iso deviennent les noms de colonnes
    df.columns = df.iloc[-1,:]
    df.columns.name = None
    # on drop les 5 premières lignes du dataset ainsi que la dernière
    df = df.drop(df.index[:6]).drop(df.index[-1])
    # on parse les dates en datetime
    df["date"] = pd.to_datetime(df["date"], dayfirst=True)
    # on remplace les , par . et le - par NaN pour pouvoir transformer les taux de change en float
    df.iloc[:, 1:] = df.iloc[:, 1:].replace({r",": ".", r"\-": np.nan}, regex=True).astype(float)
    # on utilise les dates comme index
    df = df.set_index("date")
    # on commence par fill les NaN avec la methode bfill (des dates les + anciennes vers les + récentes)
    # s'il reste des NaN on fill en forward fill (du + récent au + ancien)
    df = df.fillna(method='bfill', axis=0).fillna(method='ffill', axis=0)
    
    return df.loc[:, devises]

### Tests

In [5]:
import unittest

class Lesson4Tests(unittest.TestCase):
    def test_df_names_us(self):
        df = df_names_us()
        # colonnes
        self.assertEqual(list(df.columns), ['year', 'name', 'gender', 'births'])
        # lignes
        self.assertEqual(len(df), 2020863)
        # index
        self.assertTrue(isinstance(df.index, pd.core.indexes.range.RangeIndex))
        # test NaN
        self.assertTrue(df.loc[df.isnull().any(axis=1)].empty)
        
    def test_df_names_fr(self):
        df = df_names_fr()
        # colonnes
        self.assertEqual(list(df.columns), ['year', 'name', 'gender', 'births'])
        # lignes
        self.assertEqual(len(df), 630407)
        # index
        self.assertTrue(isinstance(df.index, pd.core.indexes.range.RangeIndex))
        # test names
        self.assertTrue(df.loc[df['name'].str.contains('^[A-Z]+(?:-[A-Z]+)?$')].empty)
        # test gender
        self.assertEqual(len(df), len(df.loc[df['gender']=='F']) + len(df.loc[df['gender']=='M']))
        # test NaN
        self.assertTrue(df.loc[df.isnull().any(axis=1)].empty)

    def test_df_taux_change(self):
        df = df_taux_change(['CHF', 'GBP', 'USD'])
        # colonnes
        self.assertEqual(list(df.columns), ['CHF', 'GBP', 'USD'])
        # index
        self.assertTrue(isinstance(df.index, pd.core.indexes.datetimes.DatetimeIndex))
        # types taux
        self.assertTrue((df.dtypes == 'float').all())
        # test NaN
        self.assertTrue(df.loc[df.isnull().any(axis=1)].empty)


In [6]:
# run tests
def run_tests():
    test_suite = unittest.makeSuite(Lesson4Tests)
    runner = unittest.TextTestRunner(verbosity=2)
    runner.run(test_suite)

In [7]:
# run tests
run_tests()

test_df_names_fr (__main__.Lesson4Tests) ... ok
test_df_names_us (__main__.Lesson4Tests) ... ok
test_df_taux_change (__main__.Lesson4Tests) ... ok

----------------------------------------------------------------------
Ran 3 tests in 9.362s

OK
